# *SISTEMA COMANDO'S* HISTORY API ENDPOINT

In [119]:
import requests, json, time, pandas as pd, numpy as np
from datetime import datetime
from matplotlib.path import Path as mpl_path
from IPython.display import clear_output as co
from google.cloud import bigquery
from google.oauth2 import service_account

comando_api = 'https://api.dados.rio/v2/adm_cor_comando'
comando_api_v1 = 'https://api.dados.rio/v1/cor/comando'
events_url = comando_api + '/ocorrencias'
events_url_v1 = comando_api_v1 + '/eventos'
pops_url = comando_api + '/pops'

# Load pops data
pops_df = pd.DataFrame(requests.get(pops_url).json()['objeto'])
# pops_df = pd.read_csv('static/comando/pops.csv')
pops_map = pops_df.set_index('id')['titulo'].to_dict()

### Comando API request functions

In [122]:
def dict_df_map(obj):
    if 'eventos' in obj.keys():
        return pd.DataFrame((obj['eventos']))
    else:
        return pd.DataFrame([obj])

def build_rqst_params(start, now, offset, url):
    n_requests = int(np.ceil((now - start) / (offset)))
    rqst_params = []
    for n in range(n_requests):
        events_res = {}
        if n != 0: start = start + offset
        end = start + offset
        start_str = start.isoformat().replace('T', ' ') + '.0'
        end_str = end.isoformat().replace('T', ' ') + '.0'
        rqst_params.append({
            'start': start_str,
            'end': end_str,
            'url': url + f'?inicio={start_str}&fim={end_str}',
        })
    return pd.DataFrame(rqst_params)

def get_comando_api_history(start, now, offset, url):
    rqst_params = build_rqst_params(start, now, offset, url)
    n_requests = int(np.ceil((now - start) / (offset)))
    events_df, events_res = [], {}
    missing = False
    for idx, param in rqst_params.iterrows():
        rqst_url = param['url']
        j = 0
        while 'eventos' not in events_res.keys():
            j+=1; co(True); print('Periods requested:', idx + 1, '/', n_requests,'\nRetries:', str(j), '/', n_attempts)
            events_res = requests.get(rqst_url).json()
            if j >= n_attempts:
                missing = True; break
        events_df.append(pd.concat(list(map(dict_df_map, events_res['eventos']))))
        events_res = {}
    events_df = pd.concat(events_df).reset_index(drop=True)
    events_df['inicio'] = events_df['inicio'].astype('datetime64[ms]')
    events_df['fim'] = events_df['fim'].astype('datetime64[ms]')
    status = 'Failed! Records missing.' if missing else 'Success.'
    print(f'Done! Status: {status}')
    return events_df

---

# Request and save Sistema Comando history from Comando API V1

## Request parameters v2

In [13]:
start = datetime(2018, 1, 1)
now = datetime.now()
offset = pd.offsets.Day() * 30
n_attempts = 10

start.isoformat(), offset

('2018-01-01T00:00:00', <30 * Days>)

In [21]:
comando_history = get_comando_api_history(start, now, offset, url=events_url)

# data processing
comando_history['pop_titulo'] = comando_history['pop_id'].map(pops_map) # include event pop title

Periods requested: 61 / 61 
Retries: 2 / 10
Done! Status: Success


### Save history from api

In [88]:
# comando_history.to_csv('../Dados/Comando/comando_history_from_api.csv', index=False)

### Check duplicates

In [98]:
n_duplicates = len(comando_history['id']) - len(comando_history['id'].unique())

if n_duplicates == 0:
    print('No duplicate keys found!')
else:
    print('Duplicates found:', n_duplicates)

Duplicates found: 68


### Reload & See result

In [94]:
comando_history = pd.read_csv('../Dados/Comando/comando_history_from_api.csv')

comando_history

,tipo,pop_id,latitude,inicio,titulo,fim,aviso_id,descricao,informe_id,gravidade,id,longitude,status,bairro,prazo,pop_titulo
0,PRIMARIO,22.0,-22.943624,2018-01-01 01:52:24,Atropelamento VP,2018-01-01 02:44:15,6482.0,Viaduto San Tiago Dantas - Botafogo,23250,BAIXO,23249,-43.181733,FECHADO,NaN,NaN,Atropelamento
1,PRIMARIO,2.0,-22.862903,2018-01-01 05:56:11,Queda MT,2018-01-01 06:54:06,6483.0,Av. Brasil - Maré,23251,BAIXO,23250,-43.247619,FECHADO,NaN,NaN,Acidente com vítima(s)
2,PRIMARIO,1.0,-22.929546,2018-01-01 11:57:10,AC VP x Canaleta,2018-01-01 12:55:22,6484.0,Estr. dos Três Rios - Km 06 - JPA,23252,BAIXO,23251,-43.322075,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
3,PRIMARIO,1.0,-22.885512,2018-01-01 12:30:04,Tombamento de CM,2018-01-01 14:42:09,0.0,Av. Marechal Fontenele - Alt. Transolimpica,23253,BAIXO,23252,-43.398607,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
4,PRIMARIO,1.0,-23.012643,2018-01-01 15:32:31,AC 3 VPs,2018-01-01 17:26:18,6486.0,Av. Lúcio Costa - Alt. Praia da Reserva - Barr...,23254,BAIXO,23253,-43.389637,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62711,PRIMARIO,1.0,-22.982936,2022-12-21 21:19:27,Enguiço ON,2022-12-22 01:37:15,NaN,Av. Ayrton Senna - Alt. Ponte Santos Dumont -...,86808,MEDIO,86807,-43.365378,FECHADO,Barra da Tijuca,CURTO,Acidente/enguiço sem vítima
62712,PRIMARIO,1.0,-22.816247,2022-12-21 22:17:12,AC CM x Divisória de Pista,2022-12-22 01:38:24,NaN,Av. Brasil - Alt. Vigore - Parada de Lucas - ...,86813,BAIXO,86812,-43.303567,FECHADO,Parada de Lucas,CURTO,Acidente/enguiço sem vítima
62713,PRIMARIO,5.0,-22.898978,2022-12-22 00:52:29,Bolsão d'água em via,2022-12-22 00:54:04,NaN,Estr. do Catonho - Alt. Jardim Sulacap - Ambo...,86816,BAIXO,86815,-43.389128,FECHADO,Jardim Sulacap,CURTO,Bolsão d'água em via
62714,PRIMARIO,28.0,-22.904143,2022-12-22 01:09:00,Rolamento de Pedra,2022-12-22 01:28:00,NaN,Rua Alzira Valdetaro - Alt. Faculdade Celso Li...,86817,BAIXO,86816,-43.263208,FECHADO,Sampaio,CURTO,Deslizamento


---

# Request and save Sistema Comando history from Comando API V1

In [113]:
start = datetime(2018, 1, 1)
now = datetime.now()
offset = pd.offsets.Day() * 30
n_attempts = 10

start.isoformat(), offset

('2018-01-01T00:00:00', <30 * Days>)

In [123]:
comando_history = get_comando_api_history(start, now, offset, url=events_url_v1)

# data processing
comando_history['pop_titulo'] = comando_history['pop_id'].map(pops_map) # include event pop title

Periods requested: 61 / 61 
Retries: 1 / 10
Done! Status: Success.


### Save history from api

In [88]:
# comando_history.to_csv('../Dados/Comando/comando_history_from_api.csv', index=False)

### Check duplicates

In [98]:
n_duplicates = len(comando_history['id']) - len(comando_history['id'].unique())

if n_duplicates == 0:
    print('No duplicate keys found!')
else:
    print('Duplicates found:', n_duplicates)

Duplicates found: 68


### Reload & See result

In [94]:
comando_history = pd.read_csv('../Dados/Comando/comando_history_from_api.csv')

comando_history

,tipo,pop_id,latitude,inicio,titulo,fim,aviso_id,descricao,informe_id,gravidade,id,longitude,status,bairro,prazo,pop_titulo
0,PRIMARIO,22.0,-22.943624,2018-01-01 01:52:24,Atropelamento VP,2018-01-01 02:44:15,6482.0,Viaduto San Tiago Dantas - Botafogo,23250,BAIXO,23249,-43.181733,FECHADO,NaN,NaN,Atropelamento
1,PRIMARIO,2.0,-22.862903,2018-01-01 05:56:11,Queda MT,2018-01-01 06:54:06,6483.0,Av. Brasil - Maré,23251,BAIXO,23250,-43.247619,FECHADO,NaN,NaN,Acidente com vítima(s)
2,PRIMARIO,1.0,-22.929546,2018-01-01 11:57:10,AC VP x Canaleta,2018-01-01 12:55:22,6484.0,Estr. dos Três Rios - Km 06 - JPA,23252,BAIXO,23251,-43.322075,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
3,PRIMARIO,1.0,-22.885512,2018-01-01 12:30:04,Tombamento de CM,2018-01-01 14:42:09,0.0,Av. Marechal Fontenele - Alt. Transolimpica,23253,BAIXO,23252,-43.398607,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
4,PRIMARIO,1.0,-23.012643,2018-01-01 15:32:31,AC 3 VPs,2018-01-01 17:26:18,6486.0,Av. Lúcio Costa - Alt. Praia da Reserva - Barr...,23254,BAIXO,23253,-43.389637,FECHADO,NaN,NaN,Acidente/enguiço sem vítima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62711,PRIMARIO,1.0,-22.982936,2022-12-21 21:19:27,Enguiço ON,2022-12-22 01:37:15,NaN,Av. Ayrton Senna - Alt. Ponte Santos Dumont -...,86808,MEDIO,86807,-43.365378,FECHADO,Barra da Tijuca,CURTO,Acidente/enguiço sem vítima
62712,PRIMARIO,1.0,-22.816247,2022-12-21 22:17:12,AC CM x Divisória de Pista,2022-12-22 01:38:24,NaN,Av. Brasil - Alt. Vigore - Parada de Lucas - ...,86813,BAIXO,86812,-43.303567,FECHADO,Parada de Lucas,CURTO,Acidente/enguiço sem vítima
62713,PRIMARIO,5.0,-22.898978,2022-12-22 00:52:29,Bolsão d'água em via,2022-12-22 00:54:04,NaN,Estr. do Catonho - Alt. Jardim Sulacap - Ambo...,86816,BAIXO,86815,-43.389128,FECHADO,Jardim Sulacap,CURTO,Bolsão d'água em via
62714,PRIMARIO,28.0,-22.904143,2022-12-22 01:09:00,Rolamento de Pedra,2022-12-22 01:28:00,NaN,Rua Alzira Valdetaro - Alt. Faculdade Celso Li...,86817,BAIXO,86816,-43.263208,FECHADO,Sampaio,CURTO,Deslizamento


---

# Request and save Sistema Comando history from BigQuery datario datalake

### BigQuery request function

In [39]:
project_id = 'pluvia-360323'
google_credentials = '../../../../Apps/Python/bolsao-api/credentials/pluvia-360323-eae2907a9c98.json'
google_credentials = service_account.Credentials.from_service_account_file(google_credentials)

def bigquery_request(query, params=[]):
    client = bigquery.Client(credentials=google_credentials)
    query_job = client.query(query.format(params))
    return pd.DataFrame(list(map(dict, query_job.result())))

query = '''
SELECT * FROM `datario.adm_cor_comando.ocorrencias` WHERE data_particao >= "2018-01-01"
'''

comando_history = bigquery_request(query, params=[])

# data processing
comando_history['pop_titulo'] = comando_history['id_pop'].map(pops_map) # include event pop title

### Save history from bigquery

In [91]:
comando_history.to_csv('../Dados/Comando/comando_history_from_bq.csv', index=False)

### Check duplicates

In [103]:
n_duplicates = len(comando_history['id_evento']) - len(comando_history['id_evento'].unique())

if n_duplicates == 0:
    print('No duplicate keys found!')
else:
    print('Duplicates found:', n_duplicates)

Duplicates found: 1022


### Reload & See result

In [102]:
comando_history = pd.read_csv('../Dados/Comando/comando_history_from_bq.csv')

comando_history

,id_pop,id_evento,bairro,data_inicio,data_fim,prazo,descricao,gravidade,latitude,longitude,status,tipo,data_particao,pop_titulo
0,1.0,79315,Pechincha,2022-08-13 04:31:21+00:00,2022-08-13 06:48:14+00:00,Curto,"Rua edgard werneck, 1633 (alt. r. abraão) - pe...",Baixo,-22.950938,-43.358698,Fechado,Primario,2022-08-13,Acidente/enguiço sem vítima
1,1.0,79314,Centro,2022-08-13 04:24:36+00:00,2022-08-13 07:12:10+00:00,Curto,"Av. pres. vargas, alt. av. passos - centro - s...",Baixo,-22.900883,-43.177842,Fechado,Primario,2022-08-13,Acidente/enguiço sem vítima
2,1.0,79318,Vila isabel,2022-08-13 08:01:06+00:00,2022-08-13 09:09:35+00:00,Curto,R. maxwell - alt. r. gonzaga bastos - vila isabel,Baixo,-22.918881,-43.241056,Fechado,Primario,2022-08-13,Acidente/enguiço sem vítima
3,1.0,79134,Recreio dos bandeirantes,2022-08-10 06:20:10+00:00,2022-08-10 06:31:33+00:00,Curto,"Av. das américas, alt. amoedo - recreio dos ba...",Baixo,-23.016001,-43.516339,Fechado,Primario,2022-08-10,Acidente/enguiço sem vítima
4,1.0,79135,Bonsucesso,2022-08-10 06:28:26+00:00,2022-08-10 07:42:57+00:00,Curto,"Av. brasil, alt. pasarela 9 - bonsucesso - sen...",Baixo,-22.856547,-43.247944,Fechado,Primario,2022-08-10,Acidente/enguiço sem vítima
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56224,1.0,45006,Engenho novo,2019-08-03 12:55:28+00:00,2019-08-03 14:05:46+00:00,Curto,Estr. grajaú jpa - km 3 - engenho novo - sent....,Baixo,-22.921105,-43.285024,Fechado,Primario,2019-08-03,Acidente/enguiço sem vítima
56225,1.0,45014,Engenho novo,2019-08-03 15:12:17+00:00,2019-08-03 15:51:38+00:00,Curto,"Estr. grajaú jpa - km 2,5 - engenho novo - sen...",Baixo,-22.921105,-43.285024,Fechado,Primario,2019-08-03,Acidente/enguiço sem vítima
56226,1.0,45019,Engenho novo,2019-08-03 16:25:22+00:00,2019-08-03 16:47:04+00:00,Curto,Estr. grajaú jpa - km 5 - engenho novo - sent....,Baixo,-22.921105,-43.285024,Fechado,Primario,2019-08-03,Acidente/enguiço sem vítima
56227,1.0,45022,Engenho novo,2019-08-03 17:27:27+00:00,2019-08-03 17:33:58+00:00,Curto,Estr. grajaú jpa -km 3 - engenho novo - sent. ...,Baixo,-22.921105,-43.285024,Fechado,Primario,2019-08-03,Acidente/enguiço sem vítima
